In [2]:
%run ../common/common.ipynb

Note: you may need to restart the kernel to use updated packages.


Caution: Nodes should have been already in `cluster.yaml`

In [15]:
run_command("kind delete cluster", shell=False)
run_command("kind create cluster --config=cluster.yaml", shell=False)
run_command("sleep 30", shell=False)

kind delete cluster
kind delete cluster
Log 0 - 2025-07-09 10:31:05 : Deleting cluster "kind" ...
Log 0 - 2025-07-09 10:31:11 : Deleted nodes: ["kind-control-plane" "kind-worker" "kind-worker3" "kind-worker2"]
kind create cluster --config=cluster.yaml
kind create cluster --config=cluster.yaml
Log 0 - 2025-07-09 10:31:11 : Creating cluster "kind" ...
Log 0 - 2025-07-09 10:31:11 : • Ensuring node image (kindest/node:v1.33.1) 🖼  ...
Log 0 - 2025-07-09 10:31:11 : ✓ Ensuring node image (kindest/node:v1.33.1) 🖼
Log 0 - 2025-07-09 10:31:11 : • Preparing nodes 📦 📦 📦 📦   ...
Log 0 - 2025-07-09 10:31:17 : ✓ Preparing nodes 📦 📦 📦 📦
Log 0 - 2025-07-09 10:31:17 : • Writing configuration 📜  ...
Log 0 - 2025-07-09 10:31:17 : ✓ Writing configuration 📜
Log 0 - 2025-07-09 10:31:17 : • Starting control-plane 🕹️  ...
Log 0 - 2025-07-09 10:31:34 : ✓ Starting control-plane 🕹️
Log 0 - 2025-07-09 10:31:34 : • Installing CNI 🔌  ...
Log 0 - 2025-07-09 10:31:35 : ✓ Installing CNI 🔌
Log 0 - 2025-07-09 10:31:35 : 

0

In [16]:
run_command('../common/common_modules.sh')



../common/common_modules.sh
../common/common_modules.sh
Log 0 - 2025-07-09 10:32:13 : clusterrolebinding.rbac.authorization.k8s.io/manager-full created
Log 0 - 2025-07-09 10:32:15 : customresourcedefinition.apiextensions.k8s.io/certificaterequests.cert-manager.io created
Log 0 - 2025-07-09 10:32:15 : customresourcedefinition.apiextensions.k8s.io/certificates.cert-manager.io created
Log 0 - 2025-07-09 10:32:15 : customresourcedefinition.apiextensions.k8s.io/challenges.acme.cert-manager.io created
Log 0 - 2025-07-09 10:32:16 : customresourcedefinition.apiextensions.k8s.io/clusterissuers.cert-manager.io created
Log 0 - 2025-07-09 10:32:16 : customresourcedefinition.apiextensions.k8s.io/issuers.cert-manager.io created
Log 0 - 2025-07-09 10:32:16 : customresourcedefinition.apiextensions.k8s.io/orders.acme.cert-manager.io created
Log 0 - 2025-07-09 10:32:16 : namespace/cert-manager created
Log 0 - 2025-07-09 10:32:16 : serviceaccount/cert-manager-cainjector created
Log 0 - 2025-07-09 10:32:1

0

In [17]:
(manager_node, jobmanager_node, taskmanager_nodes) = get_label_nodes(ip_address=True)

kind-control-plane	{'beta.kubernetes.io/arch': 'amd64', 'beta.kubernetes.io/os': 'linux', 'kubernetes.io/arch': 'amd64', 'kubernetes.io/hostname': 'kind-control-plane', 'kubernetes.io/os': 'linux', 'node-role.kubernetes.io/control-plane': '', 'node.kubernetes.io/exclude-from-external-load-balancers': ''}
kind-worker	{'beta.kubernetes.io/arch': 'amd64', 'beta.kubernetes.io/os': 'linux', 'ingress-ready': 'true', 'kubernetes.io/arch': 'amd64', 'kubernetes.io/hostname': 'kind-worker', 'kubernetes.io/os': 'linux', 'tier': 'manager'}
kind-worker2	{'beta.kubernetes.io/arch': 'amd64', 'beta.kubernetes.io/os': 'linux', 'kubernetes.io/arch': 'amd64', 'kubernetes.io/hostname': 'kind-worker2', 'kubernetes.io/os': 'linux', 'tier': 'jobmanager'}
kind-worker3	{'beta.kubernetes.io/arch': 'amd64', 'beta.kubernetes.io/os': 'linux', 'kubernetes.io/arch': 'amd64', 'kubernetes.io/hostname': 'kind-worker3', 'kubernetes.io/os': 'linux', 'tier': 'taskmanager'}


In [18]:
!kubectl describe ing -n manager

Name:             grafana
Labels:           <none>
Namespace:        manager
Address:          localhost
Ingress Class:    <none>
Default backend:  <default>
Rules:
  Host                        Path  Backends
  ----                        ----  --------
  grafana.127-0-0-1.sslip.io  
                              /   prom-grafana:80 (10.244.3.4:3000)
Annotations:                  kubernetes.io/ingress.class: nginx
                              nginx.ingress.kubernetes.io/proxy-body-size: 0
Events:
  Type    Reason  Age              From                      Message
  ----    ------  ----             ----                      -------
  Normal  Sync    4s (x2 over 4s)  nginx-ingress-controller  Scheduled for sync


Name:             prometheus
Labels:           <none>
Namespace:        manager
Address:          localhost
Ingress Class:    <none>
Default backend:  <default>
Rules:
  Host                           Path  Backends
  ----                           ----  --------
  prometheus

In [19]:
address = "127-0-0-1"
print("Manager node: {}:30080".format(manager_node))
print("Access to Minio: http://minio.{}.nip.io:30080".format(address))
print("Access to Grafana: http://grafana.{}.nip.io:30080".format(address))
print("Access to Prometheus: http://prometheus.{}:30080".format(address))
print("Job manager address: {}".format(jobmanager_node))
print("Task manager addresses: {}".format(taskmanager_nodes))

Manager node: 172.18.0.4:30080
Access to Minio: http://minio.127-0-0-1.nip.io:30080
Access to Grafana: http://grafana.127-0-0-1.nip.io:30080
Access to Prometheus: http://prometheus.127-0-0-1:30080
Job manager address: 172.18.0.5
Task manager addresses: 172.18.0.3


In [31]:
print("Access to Minio: http://{}:30900".format(manager_node))
print("Access to Grafana: http://{}:30300".format(manager_node))
print("Access to Prometheus: http://{}:30090".format(manager_node))
print("Job manager address: {}".format(jobmanager_node))
print("Task manager addresses: {}".format(taskmanager_nodes))

Access to Minio: http://172.18.0.4:30900
Access to Grafana: http://172.18.0.4:30300
Access to Prometheus: http://172.18.0.4:30090
Job manager address: 172.18.0.5
Task manager addresses: 172.18.0.3


Import [Flink dashboard](https://grafana.com/grafana/dashboards/14911) in Grafana.

In [ ]:
run_command('../minio/bucket.sh')


../minio/bucket.sh
../minio/bucket.sh
Log 0 - 2025-07-09 13:11:56 : namespace/minio-tenant unchanged
Log 0 - 2025-07-09 13:11:56 : secret/storage-configuration configured
Log 0 - 2025-07-09 13:11:56 : secret/storage-user unchanged
Log 0 - 2025-07-09 13:11:58 : tenant.minio.min.io/myminio unchanged
Log 0 - 2025-07-09 13:12:48 : Forwarding from 127.0.0.1:9000 -> 9000
Log 0 - 2025-07-09 13:12:48 : Forwarding from [::1]:9000 -> 9000
Log 0 - 2025-07-09 13:12:58 : Handling connection for 9000
Log 0 - 2025-07-09 13:12:58 : Added `myminio` successfully.
Log 0 - 2025-07-09 13:12:58 : Handling connection for 9000
Log 0 - 2025-07-09 13:12:58 : mc: <ERROR> Unable to make bucket `myminio/mybucket`. Your previous request to create the named bucket succeeded and you already own it.
